In [ ]:
import requests as rq
import datetime
import math
API_KEY = "0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z"
headers = {
    "x-api-key": API_KEY
}

In [ ]:
match_id = 103936903249772877
game_id = match_id + 1
params = dict(
    # startingTime=mydate_final,
)

r = rq.get(
    url=f"https://feed.lolesports.com/livestats/v1/window/{game_id}",
    headers=headers,
    params=params
)
r.json()
# frames = r.json()["frames"]
# frame = frames[-1]
# times = [f["rfc460Timestamp"] for f in frames]
# time = times[-1]
# time

In [ ]:
frame_list = [frame]
while frame and frame["gameState"] == "in_game":
    full_date = time.split('.')[0]
    date_time_obj = datetime.datetime.strptime(full_date, "%Y-%m-%dT%H:%M:%S")
    delta_seconds = datetime.timedelta(seconds=30)
    final_time = (date_time_obj + delta_seconds).strftime("%Y-%m-%dT%H:%M:%S")
    final_time = final_time[:-1] + "0Z"  # Floor seconds + add Z idk why
    params = dict(
        startingTime=final_time,
    )
    r2 = rq.get(
        url=f"https://feed.lolesports.com/livestats/v1/window/{game_id}",
        headers=headers,
        params=params
    )
    frame = r2.json()["frames"][-1]
    time = frame["rfc460Timestamp"]
    frame_list.append(frame)
# times2 = [f["rfc460Timestamp"] for f in frames2]
[(f["rfc460Timestamp"], f["gameState"]) for f in frame_list]

In [ ]:
def get_team_stats(team, frames):
    res = dict()
    for key in frames[0][team]:
        if key == "participants":
            res[key] = dict()
            for idx, p in enumerate(frames[0][team]["participants"]):
                p_id = p.get("participantId")
                res[key][p_id] = dict()
                for k in p:
                    if k == "participantId":
                        continue
                    res[key][p_id][k] = [f[team][key][idx][k] for f in frames]
        else:
            res[key] = [f[team][key] for f in frames]
    return res

In [ ]:

final_dict = dict()
final_dict["timestamps"] = [f["rfc460Timestamp"] for f in frame_list]
# Get blue team infos

final_dict["blueTeam"] = get_team_stats("blueTeam", frame_list)
final_dict["redTeam"] = get_team_stats("redTeam", frame_list)
final_dict["blueTeam"]["participants"].keys()

In [ ]:
import json
json.dumps(final_dict, separators=(',', ':'))